# DRBARMS algorithm

### Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
employed = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_employed.csv")  #Importing Dataset from system
employed = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_employed.csv", header=None)  #keeping header as None

fulltime = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_fulltime.csv")
fulltime = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_fulltime.csv", header=None)

parttime = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_parttime.csv")
parttime = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_parttime.csv", header=None)

unemployment = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_unemployment.csv")
unemployment = pd.read_csv(r"D:\Download\COVID19\DRBARMS_input_unemployment.csv", header=None)

#num_records = len(employed)
#print(num_records)

In [3]:
employed.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none,total_employed_medium
1,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none,total_employed_high
2,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none,total_employed_medium
3,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none,total_employed_high
4,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none,total_employed_high


### initial global count and global density

In [4]:
import itertools

def count_occurence(item_set, data):
    # we need to go over the all data, find how many times the item set is appear.
    c = []
    count = 0
    print(len(item_set))
    print(item_set)
    for i in range(len(item_set)):
        for j in range(len(data)):
            if set(item_set[i]).issubset(set(data[j])):
                count += 1
                
        #print(item_set[i])
        c.append(count)
        count = 0
        
    return c


def  init_data_processing(data):
    
    list_gc = []    # collect all series gc
    list_gd = []    # collect all series gd
    dict_gc = {}    # global count for each unique item in this Series
    dict_gd = {}    # global density for each unique item in this Series
    
   
    length = len(data.columns)
    for i in range(length):
        GC_data = data[[i]]
        GC_list = GC_data.values.tolist()
        GC_list = list(itertools.chain.from_iterable(GC_list))
        _t = list(np.unique(GC_list))    
        
        c = count_occurence(_t, GC_list)
        #print(c)
        #({k: join_itemsets(L[k-1], my_whole_set)})
        global_density = [[f/len(GC_list)] for f in c]
        #print(len(GC_list))
        
        count_flag = 0
        for item in _t:
            dict_gc.update({item : c[count_flag]})
            dict_gd.update({item : global_density[count_flag]})
            #print( global_density[count_flag])
            #print("-----")
            #print(dict_gc)
            #print(dict_gd)
            count_flag += 1
            
        list_gc.append(dict_gc)
        list_gd.append(dict_gd)
        dict_gc = {}
        dict_gd = {}
        
        #print("end")
    return list_gc, list_gd

### Get initial local count and local density & global count and global density & confidence and density ratio

In [5]:
def get_all_reason_item(data, start_index, end_index):
    local_list = []
    #print(start_index, end_index)
    for i in range(start_index, end_index):
        #print(i)
        #print(data)
        #print(data.iloc[i].values[0])
        local_list.append(data.iloc[i].values[0])
        print(data.iloc[i].values[0])
        
    return local_list 

def get_reason_item(data, width, C_data, target):
    
    item = []
    #print(data,C_data, target)
    for i in range(0,len(data),width):
    #print(i)
        for j in range(i,i+width):
            #print(C_data.iloc[j].values, target)
            #print(j)
            if j < len(data):
                print(j)
                print(C_data.iloc[j].values, target)
                if C_data.iloc[j].values == target:
                # if in a block the target item appear
                # add count for every item appear 
                
                    print("inner loop",j)
                    if i+width > len(data):
                        temp = get_all_reason_item(data,i, len(data))
                    else:
                        temp = get_all_reason_item(data,i, i+width)
                    
                    for x in range(len(temp)):
                        item.append(temp[x])  
    return item

def counting_local_density(global_count, data, global_count_target, target, width):
    
    dict_lc = {}    # local count for each unique item in this Series
    dict_ld = {}    # local density for each unique item in this Series
    
    new_t = list(global_count)
    print("* -> input check")
    print(global_count, data, global_count_target, target)
    print("* ->C local count")
    print(new_t)
    lc = count_occurence(new_t,data)
    print(lc)
    print("GC_target:")
    print(target, global_count_target)
    print("local density")
    local_density = [[f/(global_count_target[0] * width)] for f in lc]
    print(local_density)
    print("*")
    
    count_flag = 0
    for item in new_t:
        dict_lc.update({item : lc[count_flag]})
        dict_ld.update({item : local_density[count_flag]})
        count_flag += 1
        
    return dict_lc, dict_ld

def get_density_ratio(global_count, local_density, global_density):
    
    dr_dict = {}
    item_candidate = {}
    
    for item in list(global_count):
        print("density ratio:")
        print(local_density[item])
        print(global_density[item])
        dr = local_density[item][0] / global_density[item][0]
        print(item, dr)
        dr_dict.update({item: dr})    
    
        if (dr > 1):    # this can be varied. We want to collect the density ratio > 1 into relative density ratio analysis
                        # also we will use them for next round explore
                
            item_candidate.update({item:local_density[item][0]})
            print("update the item candidate",item, local_density[item][0])
        
    final_candidate = get_relative_density(item_candidate)
    
    return dr_dict, final_candidate
        

def get_relative_density(item_candidate):        
    
    final_candidate = {}

    for item in item_candidate:
        #print(len(item_candidate))
        #print(item_candidate[item])
        sum_item = 0
        for k,v in item_candidate.items():
            sum_item += v
        #print(sum_item)
        rdr = (len(item_candidate) * item_candidate[item] )/ sum_item
        print("item local_density", item_candidate[item], sum_item)
        print(item, "has a relative importance:", rdr)
        if rdr > 1:
            final_candidate.update({item: rdr})

    return final_candidate

def get_confidence(final_candidate, global_count, global_count_target, local_count):
    
    confidence = {}
    
    for k,v in final_candidate.items():
        print(k,local_count[k], global_count[k], global_count_target[0])
        conf = local_count[k] / max(global_count[k] ,global_count_target[0])
        #print(conf)
        confidence.update({k : conf})
        
    return confidence

In [6]:
def single_target_analysis(target, width, data, target_data, global_count, global_count_target, global_density):
    
    
    print(target, "as C") 
    
    item_in_data = get_reason_item(data, width, target_data, target)
    print(item_in_data)
    
    # lc : local count
    # ld : local density
    local_count, local_density = counting_local_density(global_count, item_in_data, global_count_target, target, width)
    print("the ld dict:", local_density)
    # dr : density ratio
    dr_dict, final_candidate = get_density_ratio(global_count, local_density, global_density)
    
    print("the dr dict:", dr_dict)
    print("the candidate : ", final_candidate)    

    # conf : confidence
    conf = get_confidence(final_candidate, global_count, global_count_target, local_count)
    
    print("the confidence : ", conf)
    print("===========")
    
    return final_candidate,dr_dict,conf

In [7]:
def loop_all_series(data, target, target_data, width = 2):
    
    dict_candidate_from_each_series = {}
    density_ratio_from_each_series = {}
    confidence_from_each_series = {}
    list_gc, list_gd = init_data_processing(data)
    
    # because we use the same target data, thus we initialize it at the begining.

    C = target_data   
    
    local_target_data = list(itertools.chain.from_iterable(target_data.values.tolist()))
    
    global_count_target = count_occurence(target, local_target_data)
    print(target,  global_count_target)
    
    for i in range(0, len(list_gc)):
        S = data[[i]]    # the data for all reaon series 
        global_count = list_gc[i]
        global_density = list_gd[i]
        
        print("* Exam")
        print(S)
        print(global_count)
        #print(global_density)
        final_candidate,dr_dict,conf = single_target_analysis(target, width, S, C, global_count, global_count_target, global_density)
        
        dict_candidate_from_each_series.update({i : final_candidate})
        density_ratio_from_each_series.update({i : dr_dict})
        confidence_from_each_series.update({i : conf})
        # we also want the confidence with its density, so we need to add more code here to make the function work
    
    return dict_candidate_from_each_series, density_ratio_from_each_series, confidence_from_each_series

### create data strucutre to store the initial rules

In [8]:
import pandas as pd

def create_result_dataframe(dict_new, dr_dict, conf, target):

    first_col = "antecedent"
    second_col = "consequent"
    third_col = "density ratio"
    forth_col = "confidence"
    
    first_list = []
    second_list = []
    third_list = []
    forth_list = []
    
    for k,v in dict_new.items():
        
        for i, j in v.items():
            
            first_list.append(i)
            
            second_list.append(target[0])
            #print(dr_dict[k][i])
            third_list.append(dr_dict[k][i])
            forth_list.append(conf[k][i])
    
    

    data = {first_col:  first_list,
            second_col: second_list,
            third_col: third_list,
            forth_col: forth_list
            }

    df = pd.DataFrame (data, columns = [first_col,second_col,third_col,forth_col])

    return df

### combination of candidates

In [9]:
def merge_candidate(space1, item1, space2, item2, order):
    
    temp_dict = {}
    
    if isinstance(space1, list): 
        print("your object is a list, proceeding for L > 2 ") 
        # this means the item we received will be multiple, we will have to
        # apply special check for them
        
        #print(space1)
        #print(len(space1))
        #print(space2)
        #print(len(space2))
        
        if len(space1) == len(space2):
            mark = len(space1)
            space_same_count = 0  
            print("same length")
        
            # test if the two space are only different in 1 space
            for i in range(len(space1)):
                for j in range(len(space1)):
                    if space1[i] == space2[j]:
                        space_same_count += 1
                    
            # if the difference in space is 1, we are going to check if the same space has same item
            # we need to trace the order of space
            
            if (mark-space_same_count) == 1:
                print("space allow to merge")
                temp_dict = merge_item_list(item1, item2, order)
                return temp_dict
            else:
                print("not allow to merge")
        
            
        print("====end====")
        
    else: 
        print("your object is not a list, this is the initial L2") 
    
        if space1 != space2:
            temp_dict.update({space1 : item1})
            temp_dict.update({space2 : item2})

        print(temp_dict)
    return temp_dict 


def merge_item_list(item1, item2, order):
    length = len(item1)
    count = 0
    same_space = []
    temp = item1.copy()
    temp2 = item2.copy()
    print(item1)
    print(item2)
    for i in order:
        if (i in item1) & (i in item2):
            if item1[i] == item2[i]:
                count += 1
                print(i, "is the same space")
                same_space.append(i)
    
    if (length - count) == 1:
        # we will merge item here with the support of order
        print("item allow to merge")
        for j in range(len(same_space)):
            del temp2[same_space[j]]
        
        temp.update(temp2.items())
        return temp
        
    return {}

def check_duplicate(all_rule_set, item_set):
    
    result = False
    temp = {}
    
    for i in range(len(all_rule_set)):
        count = 0
        temp = all_rule_set[i]
        mark = len(list(temp))
        
        # if there is a rule exist, has the same with item_set, we consider this is dupulicate
        
        for k,v in temp.items():
            if k in item_set :
                if v == item_set[k]:
                    count += 1
                    if count == mark:
                        result = True
    
    return result



In [10]:
def create_L2(candidate_dict, space_name_order):
    the_initial_space = list(candidate_dict)
    L2_item_set = [] # we will get a list of dictrionary
    
    for i in range(len(the_initial_space) -1 ):
        #print(the_initial_space[i])
        for j in range(i+1, len(the_initial_space)):
        
            print(i,j)
            
            first_group_candidate = list(candidate_dict[the_initial_space[i]])
            print(first_group_candidate)
            second_group_candidate = list(candidate_dict[the_initial_space[j]])
            print(second_group_candidate)
            for x in range(len(first_group_candidate)):
                for y in range(len(second_group_candidate)):
                    L2_item_set.append (merge_candidate(the_initial_space[i],first_group_candidate[x], 
                                                        the_initial_space[j], second_group_candidate[y], space_name_order))
    
    return L2_item_set


def create_Ln(candidate_list, space_name_order):

    Ln = []
    for i in range(0,len(candidate_list)-1):
        for j in range(i+1,len(candidate_list)):
            #print(list(L2_item_set[i]), list(L2_item_set[j]))
            #print(L2_item_set[i], L2_item_set[j])

            item_set = merge_candidate(list(candidate_list[i]),candidate_list[i], list(candidate_list[j]), candidate_list[j],space_name_order)
            print(item_set)

            if len(item_set) > 0:
                # we need to check if duplicate
                if check_duplicate(Ln,item_set):
                    print("duplicate")
                else:
                    Ln.append(item_set)
                    
    return Ln

### get item sets global count, global density, local count, local density, density ratio, confidence level

In [11]:
def get_item_set_count(item_set, data):
    
    # we need to go over the all data, find how many times the item set is appear.
    # because we want to deal with the count of item set, we need to use different logic
    
    # here we received a muliple item set, and a data.
    # we are going to count how many times it appear
    # thus we return number

        
    count = 0
    space = list(item_set)
    mark = len(space)
    flag = 0
    
    for i in range(len(data)):
        for k,v in item_set.items():
            if data[k][[i]].values[0] == v:
                #print(data[k][[i]].values[0])
                flag += 1
        
        if mark == flag:
            #print("item match in", i)
            count += 1
            flag = 0
        else:
            flag = 0
    
    print(count)
    return count

def get_gc_gd_lc_ld_for_item_set(item_set, data , width = 2, target = ""):
    
    list_space = []
    list_gc = []
    list_item = []
    list_gd = []
    list_lc = []
    list_ld = []
    list_dr = []
    list_conf = []
    # get global count
    # also the local count
    for i in range(len(item_set)):
        #print(item_set[i])
        temp = []
        for k,v in item_set[i].items():
            temp.append(v)
            list_space.append(list(item_set[i]))
            list_item.append(temp)
            list_gc.append(get_item_set_count(item_set[i],data))
    
    # get global density
    list_gd = [[f/len(data)] for f in list_gc]
    print(list_gd)
    
    # get local count & local density
    if isinstance(target, list):
        # originally, the target was been passed here and process for all
        # however now we only pass one target in a list format to process
        print("target ",target, "is list")
        target = target[0]
        for i in range(len(item_set)):
            local_item_set_count, global_target = count_item_set_in_window(data, width, item_set[i], target)
            list_lc.append(local_item_set_count)
            print(local_item_set_count)
            if global_target >0:            
                list_ld.append(local_item_set_count / (width*global_target))
            else: 
                list_ld.append(0)
                print(list_gd[i])
            if list_gd[i][0] == 0:
                list_dr.append(0)  
                list_conf.append(0)
            else:
                list_dr.append((local_item_set_count / (width*global_target)) / list_gd[i][0])
                list_conf.append(local_item_set_count / max(list_gc[i],global_target) )
                 
        
    # we calculate the next round cancadidate and relative density ratio here
    result = []
    candidate_index = []
    sum_item = 0     
    for i in range(len(list_dr)):
        if list_dr[i] > 1:
            print("A:",len(list_space) )
            print(list_space[i])
            print("B:", len(list_item))
            print(list_item[i])
            print("C:", len(list_dr))
            print(list_dr[i])
            print("D:", len(list_conf))
            print(list_conf[i])
            if list_conf[i] > 0.1:
            
                temp = [list_space[i], list_item[i],[target],list_dr[i] ,list_conf[i]]
                result.append(temp)
                candidate_index.append(i)
                sum_item += list_ld[i]  #  for sum of significant ld , used in rdr calculation
                        
            print("space:")
            print(list_space[i])
            print("item:")
            print(list_item[i])
            print("density ratio:")
            print(list_dr[i])
            print("confidence:")
            print(list_conf[i])
    
    final_candidate = []
    print("creating the final candidate")
    print("the index", candidate_index)
    for i in range(len(candidate_index)):   
        rdr = (len(candidate_index) * list_ld[candidate_index[i]] )/ sum_item
        print(list_item[candidate_index[i]], "rdr:", rdr)
        print(list_space[candidate_index[i]], "rdr:", rdr)
        if rdr > 1:
            temp_candidate = {}
            for j in range(len(list_item[candidate_index[i]])):
                temp_candidate.update({list_space[candidate_index[i]][j] : list_item[candidate_index[i]][j]})
            
            final_candidate.append(temp_candidate)
    
    print(final_candidate)
    
    return result, final_candidate

def count_item_set(data, item_set, start_index, end_index):
    c = 0
    local_data = data[start_index:end_index].copy()
    local_data.reset_index(drop=True, inplace=True)
    #print("local_data", local_data)
    c = get_item_set_count(item_set, local_data)
        
    return c 

def count_item_set_in_window(data, width, item_set, target):
    
    count = 0
    target_count = 0
    length = int(len(data.index))
    #print(length)
    #print(width)
    for i in range(0,length ,width):
        for j in range(i,i+width):
            if j < length:
                if data[len(data.columns) -1][j] == target:
                    # if in a block the target item appear
                    # add count for this item appear 
                    target_count += 1
                    #print(data[len(data.columns) -1][j])
                    #print("window:", i)
                    count += count_item_set(data, item_set, i, i+width)
    return count,target_count

In [12]:
def create_item_set_dataframe(result):
    
    if len(result) > 0:
        first_col = "antecedent"
        second_col = "consequent"
        third_col = "density ratio"
        forth_col = "confidence"
    
        first_list = []
        second_list = []
        third_list = []
        forth_list = []
    
        for i in range(len(result)):
            
                first_list.append(result[i][1])
                second_list.append(result[i][2][0])
                #print("$$$$$$$$$$$$$")
                print (result[i][2][0])
                #print("$$$$$$$$$$$$$")
                third_list.append(result[i][3])
                forth_list.append(result[i][4])
    
        data = {first_col:  first_list,
                second_col: second_list,
                third_col: third_list,
                forth_col: forth_list
                }

        df = pd.DataFrame (data, columns = [first_col,second_col,third_col,forth_col])
        
        return df

    return []
    

## Complete function which deal with all possible target

In [13]:
def DRBARMS_algorithm(store_data):
    mark = len(store_data.columns)
    cleaned_data = store_data[[x for x in range(mark-1)]]
    target_data = store_data[[mark - 1]]
    print(mark)
    print(target_data)

    space_name_order = cleaned_data.columns.values

    #test_target = ["Meat"] 
    target_list = target_data.values.tolist() 
    target_list = list(itertools.chain.from_iterable(target_list))
    test_target = list(np.unique(target_list)) 

    df = pd.DataFrame(columns = ["antecedent", "consequent", "density ratio", "confidence"])

    for item_test in test_target:
        print("target:", item_test)
        item_test = [item_test]
        # we do all target in loop
        # and we use recursion to extract all information
        dict_new, dr_dict, conf = loop_all_series(cleaned_data, item_test, target_data)
        df_init = create_result_dataframe(dict_new, dr_dict, conf, item_test)
        df = pd.concat([df, df_init])
        
        print(dict_new)
        L2_item_set = create_L2(dict_new, space_name_order)
        result, candidate = get_gc_gd_lc_ld_for_item_set(L2_item_set,store_data,2, item_test)
    
        #loop
        flag = True
        while flag:
            if len(candidate) > 0:
                if len(result) > 0:
                    df_next = create_item_set_dataframe(result)
                    df = pd.concat([df, df_next])
                    # do a inner recursion
                    Lncandidate = create_Ln(candidate, space_name_order)
                    result, candidate = get_gc_gd_lc_ld_for_item_set(Lncandidate,store_data,2, item_test)
                
            else:
                if len(result) > 0:
                    df_next = create_item_set_dataframe(result)
                    df = pd.concat([df, df_next])
                
                flag = False
    
    return df

In [14]:
employed = DRBARMS_algorithm(employed)

19
                            18
0        total_employed_medium
1          total_employed_high
2        total_employed_medium
3          total_employed_high
4          total_employed_high
5          total_employed_high
6          total_employed_high
7          total_employed_high
8          total_employed_high
9          total_employed_high
10         total_employed_high
11    total_employed_very_high
12         total_employed_high
13    total_employed_very_high
14         total_employed_high
15          total_employed_low
16  total_employed_extreme_low
17     total_employed_very_low
18          total_employed_low
19          total_employed_low
20       total_employed_medium
target: total_employed_extreme_low
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
4
['Workplace_close_Recommended', '

['total_employed_high'] ['total_employed_extreme_low']
7
['total_employed_high'] ['total_employed_extreme_low']
8
['total_employed_high'] ['total_employed_extreme_low']
9
['total_employed_high'] ['total_employed_extreme_low']
10
['total_employed_high'] ['total_employed_extreme_low']
11
['total_employed_very_high'] ['total_employed_extreme_low']
12
['total_employed_high'] ['total_employed_extreme_low']
13
['total_employed_very_high'] ['total_employed_extreme_low']
14
['total_employed_high'] ['total_employed_extreme_low']
15
['total_employed_low'] ['total_employed_extreme_low']
16
['total_employed_extreme_low'] ['total_employed_extreme_low']
inner loop 16
coordinated_public_information_campaign
coordinated_public_information_campaign
17
['total_employed_very_low'] ['total_employed_extreme_low']
18
['total_employed_low'] ['total_employed_extreme_low']
19
['total_employed_low'] ['total_employed_extreme_low']
20
['total_employed_medium'] ['total_employed_extreme_low']
['coordinated_public_i

School_normal
School_normal
9
['total_employed_high'] ['total_employed_high']
inner loop 9
School_normal
School_normal
10
['total_employed_high'] ['total_employed_high']
inner loop 10
School_normal
School_normal
11
['total_employed_very_high'] ['total_employed_high']
12
['total_employed_high'] ['total_employed_high']
inner loop 12
School_normal
School_normal
13
['total_employed_very_high'] ['total_employed_high']
14
['total_employed_high'] ['total_employed_high']
inner loop 14
School_close_Required_all
School_close_Required_partly
15
['total_employed_low'] ['total_employed_high']
16
['total_employed_extreme_low'] ['total_employed_high']
17
['total_employed_very_low'] ['total_employed_high']
18
['total_employed_low'] ['total_employed_high']
19
['total_employed_low'] ['total_employed_high']
20
['total_employed_medium'] ['total_employed_high']
['School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_no

no_relie
no_relie
5
['total_employed_high'] ['total_employed_high']
inner loop 5
no_relie
no_relie
6
['total_employed_high'] ['total_employed_high']
inner loop 6
no_relie
no_relie
7
['total_employed_high'] ['total_employed_high']
inner loop 7
no_relie
no_relie
8
['total_employed_high'] ['total_employed_high']
inner loop 8
no_relie
no_relie
9
['total_employed_high'] ['total_employed_high']
inner loop 9
no_relie
no_relie
10
['total_employed_high'] ['total_employed_high']
inner loop 10
no_relie
no_relie
11
['total_employed_very_high'] ['total_employed_high']
12
['total_employed_high'] ['total_employed_high']
inner loop 12
no_relie
no_relie
13
['total_employed_very_high'] ['total_employed_high']
14
['total_employed_high'] ['total_employed_high']
inner loop 14
broad_relief
broad_relief
15
['total_employed_low'] ['total_employed_high']
16
['total_employed_extreme_low'] ['total_employed_high']
17
['total_employed_very_low'] ['total_employed_high']
18
['total_employed_low'] ['total_employed_hi

[]
0 16
[]
[]
0 17
[]
[]
1 2
[]
[]
1 3
[]
[]
1 4
[]
[]
1 5
[]
[]
1 6
[]
[]
1 7
[]
[]
1 8
[]
[]
1 9
[]
[]
1 10
[]
[]
1 11
[]
[]
1 12
[]
[]
1 13
[]
[]
1 14
[]
[]
1 15
[]
[]
1 16
[]
[]
1 17
[]
[]
2 3
[]
[]
2 4
[]
[]
2 5
[]
[]
2 6
[]
[]
2 7
[]
[]
2 8
[]
[]
2 9
[]
[]
2 10
[]
[]
2 11
[]
[]
2 12
[]
[]
2 13
[]
[]
2 14
[]
[]
2 15
[]
[]
2 16
[]
[]
2 17
[]
[]
3 4
[]
[]
3 5
[]
[]
3 6
[]
[]
3 7
[]
[]
3 8
[]
[]
3 9
[]
[]
3 10
[]
[]
3 11
[]
[]
3 12
[]
[]
3 13
[]
[]
3 14
[]
[]
3 15
[]
[]
3 16
[]
[]
3 17
[]
[]
4 5
[]
[]
4 6
[]
[]
4 7
[]
[]
4 8
[]
[]
4 9
[]
[]
4 10
[]
[]
4 11
[]
[]
4 12
[]
[]
4 13
[]
[]
4 14
[]
[]
4 15
[]
[]
4 16
[]
[]
4 17
[]
[]
5 6
[]
[]
5 7
[]
[]
5 8
[]
[]
5 9
[]
[]
5 10
[]
[]
5 11
[]
[]
5 12
[]
[]
5 13
[]
[]
5 14
[]
[]
5 15
[]
[]
5 16
[]
[]
5 17
[]
[]
6 7
[]
[]
6 8
[]
[]
6 9
[]
[]
6 10
[]
[]
6 11
[]
[]
6 12
[]
[]
6 13
[]
[]
6 14
[]
[]
6 15
[]
[]
6 16
[]
[]
6 17
[]
[]
7 8
[]
[]
7 9
[]
[]
7 10
[]
[]
7 11
[]
[]
7 12
[]
[]
7 13
[]
[]
7 14
[]
[]
7 15
[]
[]
7 16
[]
[]
7 17
[]
[]
8 9
[]
[]

...

{'School_close_Required_all': 7, 'School_close_Required_partly': 5, 'School_normal': 14} ['School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_close_Required_partly'] [3] ['total_employed_medium']
* ->C local count
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
3
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
[1, 1, 4]
GC_target:
['total_employed_medium'] [3]
local density
[[0.16666666666666666], [0.16666666666666666], [0.6666666666666666]]
*
the ld dict: {'School_close_Required_all': [0.16666666666666666], 'School_close_Required_partly': [0.16666666666666666], 'School_normal': [0.6666666666666666]}
density ratio:
[0.16666666666666666]
[0.3333333333333333]
School_close_Required_all 0.5
density ratio:
[0.16666666666666666]
[0.23809523809523808]
School_close_Required_partly 0.7
density ratio:
[0.6666666666666666]
[0.6666666666666666]
School_normal 1.0
the dr dict: {'School_close_Required_all': 0.5, 'Sc

20  Government_Response_very_high
{'Government_Response_high': 7, 'Government_Response_low': 2, 'Government_Response_none': 21, 'Government_Response_very_high': 4, 'Government_Response_very_low': 1}
['total_employed_medium'] as C
0
['total_employed_medium'] ['total_employed_medium']
inner loop 0
Government_Response_none
Government_Response_none
1
['total_employed_high'] ['total_employed_medium']
2
['total_employed_medium'] ['total_employed_medium']
inner loop 2
Government_Response_none
Government_Response_none
3
['total_employed_high'] ['total_employed_medium']
4
['total_employed_high'] ['total_employed_medium']
5
['total_employed_high'] ['total_employed_medium']
6
['total_employed_high'] ['total_employed_medium']
7
['total_employed_high'] ['total_employed_medium']
8
['total_employed_high'] ['total_employed_medium']
9
['total_employed_high'] ['total_employed_medium']
10
['total_employed_high'] ['total_employed_medium']
11
['total_employed_very_high'] ['total_employed_medium']
12
['tota

12
['total_employed_high'] ['total_employed_very_high']
13
['total_employed_very_high'] ['total_employed_very_high']
inner loop 13
stay_home_not_require
stay_home_not_require
14
['total_employed_high'] ['total_employed_very_high']
15
['total_employed_low'] ['total_employed_very_high']
16
['total_employed_extreme_low'] ['total_employed_very_high']
17
['total_employed_very_low'] ['total_employed_very_high']
18
['total_employed_low'] ['total_employed_very_high']
19
['total_employed_low'] ['total_employed_very_high']
20
['total_employed_medium'] ['total_employed_very_high']
['stay_home_not_require', 'stay_home_not_require', 'stay_home_not_require', 'stay_home_not_require']
* -> input check
{'stay_home_not_require': 20, 'stay_home_recommend': 5, 'stay_home_required_exercise_only': 4} ['stay_home_not_require', 'stay_home_not_require', 'stay_home_not_require', 'stay_home_not_require'] [2] ['total_employed_very_high']
* ->C local count
['stay_home_not_require', 'stay_home_recommend', 'stay_hom

12
12
0
0
0
0
[[0.5714285714285714], [0.5714285714285714], [0.0], [0.0], [0.0], [0.0]]
target  ['total_employed_very_high'] is list
2
0
2
0
0
0
0
0
0
creating the final candidate
the index []
[]
target: total_employed_very_low
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
4
['Workplace_close_Recommended', 'Workplace_close_Required_partly', 'Workplace_close_all_essential_only', 'Workplace_normal']
3
['Public_events_normal', 'public_events_cancel_Recommended', 'public_events_cancel_Required']
3
['No_gatherings_restrictions', 'Restrictions_ten_ppl_less', 'Restrictions_tenppl_more']
2
['transport_closing_recommend', 'transport_normal']
3
['stay_home_not_require', 'stay_home_recommend', 'stay_home_required_exercise_only']
2
['no_movement_restrictions', 'restrict_movement']
3
['border_closure', '

restrict_movement
18
['total_employed_low'] ['total_employed_very_low']
19
['total_employed_low'] ['total_employed_very_low']
20
['total_employed_medium'] ['total_employed_very_low']
['restrict_movement', 'restrict_movement']
* -> input check
{'no_movement_restrictions': 21, 'restrict_movement': 21} ['restrict_movement', 'restrict_movement'] [1] ['total_employed_very_low']
* ->C local count
['no_movement_restrictions', 'restrict_movement']
2
['no_movement_restrictions', 'restrict_movement']
[2, 2]
GC_target:
['total_employed_very_low'] [1]
local density
[[1.0], [1.0]]
*
the ld dict: {'no_movement_restrictions': [1.0], 'restrict_movement': [1.0]}
density ratio:
[1.0]
[1.0]
no_movement_restrictions 1.0
density ratio:
[1.0]
[1.0]
restrict_movement 1.0
the dr dict: {'no_movement_restrictions': 1.0, 'restrict_movement': 1.0}
the candidate :  {}
the confidence :  {}
* Exam
                       9
0      no_travel_control
1      no_travel_control
2      no_travel_control
3      no_travel_con

In [15]:
employed.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)

,antecedent,consequent,density ratio,confidence
4,stay_home_recommend,total_employed_low,2.520000,1.200000
1,"[cases_increase, comprehensive_tracing]",total_employed_low,10.500000,1.000000
2,Government_Response_low,total_employed_very_high,5.250000,1.000000
1,deaths_increase,total_employed_low,1.800000,0.857143
2,School_close_Required_all,total_employed_low,1.800000,0.857143
6,Government_Response_high,total_employed_low,1.800000,0.857143
7,ContainmentHealth_high,total_employed_low,1.800000,0.857143
3,Workplace_close_Required_partly,total_employed_low,2.100000,0.800000
0,cases_increase,total_employed_low,1.400000,0.666667
5,comprehensive_tracing,total_employed_low,1.400000,0.666667


In [16]:
#df.sort_values(by=[ 'density ratio','consequent', 'confidence'], ascending=False)
DRBARMS_result_employed = employed.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)
DRBARMS_result_employed.to_csv(r"D:\Download\COVID19\DRBARMS_result_employed.csv", index = False, header=True)

In [17]:
parttime = DRBARMS_algorithm(parttime)

19
                                18
0           Part_time_employed_low
1        Part_time_employed_medium
2          Part_time_employed_high
3          Part_time_employed_high
4     Part_time_employed_very_high
5     Part_time_employed_very_high
6          Part_time_employed_high
7     Part_time_employed_very_high
8     Part_time_employed_very_high
9          Part_time_employed_high
10    Part_time_employed_very_high
11    Part_time_employed_very_high
12       Part_time_employed_medium
13         Part_time_employed_high
14    Part_time_employed_very_high
15          Part_time_employed_low
16  Part_time_employed_extreme_low
17          Part_time_employed_low
18       Part_time_employed_medium
19         Part_time_employed_high
20         Part_time_employed_high
target: Part_time_employed_extreme_low
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Require

broad_relief
broad_relief
17
['Part_time_employed_low'] ['Part_time_employed_extreme_low']
18
['Part_time_employed_medium'] ['Part_time_employed_extreme_low']
19
['Part_time_employed_high'] ['Part_time_employed_extreme_low']
20
['Part_time_employed_high'] ['Part_time_employed_extreme_low']
['broad_relief', 'broad_relief']
* -> input check
{'broad_relief': 7, 'no_relie': 14} ['broad_relief', 'broad_relief'] [1] ['Part_time_employed_extreme_low']
* ->C local count
['broad_relief', 'no_relie']
2
['broad_relief', 'no_relie']
[2, 0]
GC_target:
['Part_time_employed_extreme_low'] [1]
local density
[[1.0], [0.0]]
*
the ld dict: {'broad_relief': [1.0], 'no_relie': [0.0]}
density ratio:
[1.0]
[0.3333333333333333]
broad_relief 3.0
update the item candidate broad_relief 1.0
density ratio:
[0.0]
[0.6666666666666666]
no_relie 0.0
item local_density 1.0 1.0
broad_relief has a relative importance: 1.0
the dr dict: {'broad_relief': 3.0, 'no_relie': 0.0}
the candidate :  {}
the confidence :  {}
* Exam
 

* Exam
                               2
0                  School_normal
1                  School_normal
2                  School_normal
3                  School_normal
4                  School_normal
5                  School_normal
6                  School_normal
7                  School_normal
8                  School_normal
9                  School_normal
10                 School_normal
11                 School_normal
12                 School_normal
13                 School_normal
14     School_close_Required_all
15  School_close_Required_partly
16  School_close_Required_partly
17  School_close_Required_partly
18  School_close_Required_partly
19     School_close_Required_all
20  School_close_Required_partly
{'School_close_Required_all': 7, 'School_close_Required_partly': 5, 'School_normal': 14}
['Part_time_employed_high'] as C
0
['Part_time_employed_low'] ['Part_time_employed_high']
1
['Part_time_employed_medium'] ['Part_time_employed_high']
2
['Part_time_employed_high'

20  open_public_testing
{'no_testing_policy': 12, 'open_public_testing': 6, 'test_selected_ppl': 3}
['Part_time_employed_high'] as C
0
['Part_time_employed_low'] ['Part_time_employed_high']
1
['Part_time_employed_medium'] ['Part_time_employed_high']
2
['Part_time_employed_high'] ['Part_time_employed_high']
inner loop 2
no_testing_policy
no_testing_policy
3
['Part_time_employed_high'] ['Part_time_employed_high']
inner loop 3
no_testing_policy
no_testing_policy
4
['Part_time_employed_very_high'] ['Part_time_employed_high']
5
['Part_time_employed_very_high'] ['Part_time_employed_high']
6
['Part_time_employed_high'] ['Part_time_employed_high']
inner loop 6
no_testing_policy
no_testing_policy
7
['Part_time_employed_very_high'] ['Part_time_employed_high']
8
['Part_time_employed_very_high'] ['Part_time_employed_high']
9
['Part_time_employed_high'] ['Part_time_employed_high']
inner loop 9
no_testing_policy
no_testing_policy
10
['Part_time_employed_very_high'] ['Part_time_employed_high']
11
['P

School_close_Required_all
School_close_Required_partly
16
['Part_time_employed_extreme_low'] ['Part_time_employed_low']
17
['Part_time_employed_low'] ['Part_time_employed_low']
inner loop 17
School_close_Required_partly
School_close_Required_partly
18
['Part_time_employed_medium'] ['Part_time_employed_low']
19
['Part_time_employed_high'] ['Part_time_employed_low']
20
['Part_time_employed_high'] ['Part_time_employed_low']
['School_normal', 'School_normal', 'School_close_Required_all', 'School_close_Required_partly', 'School_close_Required_partly', 'School_close_Required_partly']
* -> input check
{'School_close_Required_all': 7, 'School_close_Required_partly': 5, 'School_normal': 14} ['School_normal', 'School_normal', 'School_close_Required_all', 'School_close_Required_partly', 'School_close_Required_partly', 'School_close_Required_partly'] [4] ['Part_time_employed_low']
* ->C local count
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
3
['School_close_Requ

...

12
['Part_time_employed_medium'] ['Part_time_employed_medium']
inner loop 12
No_gatherings_restrictions
No_gatherings_restrictions
13
['Part_time_employed_high'] ['Part_time_employed_medium']
14
['Part_time_employed_very_high'] ['Part_time_employed_medium']
15
['Part_time_employed_low'] ['Part_time_employed_medium']
16
['Part_time_employed_extreme_low'] ['Part_time_employed_medium']
17
['Part_time_employed_low'] ['Part_time_employed_medium']
18
['Part_time_employed_medium'] ['Part_time_employed_medium']
inner loop 18
Restrictions_ten_ppl_less
Restrictions_ten_ppl_less
19
['Part_time_employed_high'] ['Part_time_employed_medium']
20
['Part_time_employed_high'] ['Part_time_employed_medium']
['No_gatherings_restrictions', 'No_gatherings_restrictions', 'No_gatherings_restrictions', 'No_gatherings_restrictions', 'Restrictions_ten_ppl_less', 'Restrictions_ten_ppl_less']
* -> input check
{'No_gatherings_restrictions': 14, 'Restrictions_ten_ppl_less': 7, 'Restrictions_tenppl_more': 1} ['No_gath

20  ContainmentHealth_very_high
{'ContainmentHealth_high': 7, 'ContainmentHealth_low': 2, 'ContainmentHealth_none': 21, 'ContainmentHealth_very_high': 4}
['Part_time_employed_medium'] as C
0
['Part_time_employed_low'] ['Part_time_employed_medium']
1
['Part_time_employed_medium'] ['Part_time_employed_medium']
inner loop 1
ContainmentHealth_none
ContainmentHealth_none
2
['Part_time_employed_high'] ['Part_time_employed_medium']
3
['Part_time_employed_high'] ['Part_time_employed_medium']
4
['Part_time_employed_very_high'] ['Part_time_employed_medium']
5
['Part_time_employed_very_high'] ['Part_time_employed_medium']
6
['Part_time_employed_high'] ['Part_time_employed_medium']
7
['Part_time_employed_very_high'] ['Part_time_employed_medium']
8
['Part_time_employed_very_high'] ['Part_time_employed_medium']
9
['Part_time_employed_high'] ['Part_time_employed_medium']
10
['Part_time_employed_very_high'] ['Part_time_employed_medium']
11
['Part_time_employed_very_high'] ['Part_time_employed_medium']

0
0
0
0
[[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.5714285714285714], [0.5714285714285714], [0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
target  ['Part_time_employed_medium'] is list
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
creating the final candidate
the index []
[]
target: Part_time_employed_very_high
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
4
['Workplace_close_Recommended', 'Workplace_close_Required_partly', 'Workplace_close_all_essential_only', 'Workplace_normal']
3
['Public_events_normal', 'public_events_cancel_Recommended', 'public_events_cancel_Required']
3
['No_gatherings_restrictions', 'Restrictions_ten_ppl_less', 'Restrictions_tenppl_more']
2
['transpo

['Part_time_employed_very_high'] ['Part_time_employed_very_high']
inner loop 7
stay_home_not_require
stay_home_not_require
8
['Part_time_employed_very_high'] ['Part_time_employed_very_high']
inner loop 8
stay_home_not_require
stay_home_not_require
9
['Part_time_employed_high'] ['Part_time_employed_very_high']
10
['Part_time_employed_very_high'] ['Part_time_employed_very_high']
inner loop 10
stay_home_not_require
stay_home_not_require
11
['Part_time_employed_very_high'] ['Part_time_employed_very_high']
inner loop 11
stay_home_not_require
stay_home_not_require
12
['Part_time_employed_medium'] ['Part_time_employed_very_high']
13
['Part_time_employed_high'] ['Part_time_employed_very_high']
14
['Part_time_employed_very_high'] ['Part_time_employed_very_high']
inner loop 14
stay_home_recommend
stay_home_required_exercise_only
15
['Part_time_employed_low'] ['Part_time_employed_very_high']
16
['Part_time_employed_extreme_low'] ['Part_time_employed_very_high']
17
['Part_time_employed_low'] ['Par

ContainmentHealth_none 1.0
density ratio:
[0.07142857142857142]
[0.19047619047619047]
ContainmentHealth_very_high 0.375
the dr dict: {'ContainmentHealth_high': 0.42857142857142855, 'ContainmentHealth_low': 0.0, 'ContainmentHealth_none': 1.0, 'ContainmentHealth_very_high': 0.375}
the candidate :  {}
the confidence :  {}
{0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}}
0 1
[]
[]
0 2
[]
[]
0 3
[]
[]
0 4
[]
[]
0 5
[]
[]
0 6
[]
[]
0 7
[]
[]
0 8
[]
[]
0 9
[]
[]
0 10
[]
[]
0 11
[]
[]
0 12
[]
[]
0 13
[]
[]
0 14
[]
[]
0 15
[]
[]
0 16
[]
[]
0 17
[]
[]
1 2
[]
[]
1 3
[]
[]
1 4
[]
[]
1 5
[]
[]
1 6
[]
[]
1 7
[]
[]
1 8
[]
[]
1 9
[]
[]
1 10
[]
[]
1 11
[]
[]
1 12
[]
[]
1 13
[]
[]
1 14
[]
[]
1 15
[]
[]
1 16
[]
[]
1 17
[]
[]
2 3
[]
[]
2 4
[]
[]
2 5
[]
[]
2 6
[]
[]
2 7
[]
[]
2 8
[]
[]
2 9
[]
[]
2 10
[]
[]
2 11
[]
[]
2 12
[]
[]
2 13
[]
[]
2 14
[]
[]
2 15
[]
[]
2 16
[]
[]
2 17
[]
[]
3 4
[]
[]
3 5
[]
[]
3 6
[]
[]
3 7
[]
[]


In [18]:
parttime.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)

,antecedent,consequent,density ratio,confidence
6,"[deaths_increase, public_events_cancel_Required]",Part_time_employed_low,5.250000,1.000000
7,"[deaths_increase, Restrictions_ten_ppl_less]",Part_time_employed_low,3.500000,0.666667
9,"[deaths_increase, gov_stringency_high]",Part_time_employed_low,3.500000,0.666667
3,Government_Response_low,Part_time_employed_medium,3.500000,0.666667
1,deaths_increase,Part_time_employed_low,1.500000,0.571429
2,School_close_Required_all,Part_time_employed_low,1.500000,0.571429
4,Restrictions_ten_ppl_less,Part_time_employed_low,1.500000,0.571429
4,Government_Response_very_high,Part_time_employed_medium,1.750000,0.500000
3,public_events_cancel_Required,Part_time_employed_low,1.312500,0.500000
5,gov_stringency_high,Part_time_employed_low,1.312500,0.500000


In [21]:
DRBARMS_result_parttime = parttime.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)
DRBARMS_result_parttime.to_csv(r"D:\Download\COVID19\DRBARMS_result_parttime.csv", index = False, header=True)

In [22]:
fulltime = DRBARMS_algorithm(fulltime)

19
                                18
0        Full_time_employed_medium
1          Full_time_employed_high
2        Full_time_employed_medium
3        Full_time_employed_medium
4        Full_time_employed_medium
5        Full_time_employed_medium
6          Full_time_employed_high
7        Full_time_employed_medium
8          Full_time_employed_high
9        Full_time_employed_medium
10         Full_time_employed_high
11    Full_time_employed_very_high
12         Full_time_employed_high
13    Full_time_employed_very_high
14         Full_time_employed_high
15          Full_time_employed_low
16     Full_time_employed_very_low
17  Full_time_employed_extreme_low
18     Full_time_employed_very_low
19     Full_time_employed_very_low
20       Full_time_employed_medium
target: Full_time_employed_extreme_low
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Require

* Exam
                     13
0     no_testing_policy
1     no_testing_policy
2     no_testing_policy
3     no_testing_policy
4     no_testing_policy
5     no_testing_policy
6     no_testing_policy
7     no_testing_policy
8     no_testing_policy
9     no_testing_policy
10    no_testing_policy
11    no_testing_policy
12    test_selected_ppl
13    test_selected_ppl
14    test_selected_ppl
15  open_public_testing
16  open_public_testing
17  open_public_testing
18  open_public_testing
19  open_public_testing
20  open_public_testing
{'no_testing_policy': 12, 'open_public_testing': 6, 'test_selected_ppl': 3}
['Full_time_employed_extreme_low'] as C
0
['Full_time_employed_medium'] ['Full_time_employed_extreme_low']
1
['Full_time_employed_high'] ['Full_time_employed_extreme_low']
2
['Full_time_employed_medium'] ['Full_time_employed_extreme_low']
3
['Full_time_employed_medium'] ['Full_time_employed_extreme_low']
4
['Full_time_employed_medium'] ['Full_time_employed_extreme_low']
5
['Full_time_em

['Full_time_employed_very_high'] ['Full_time_employed_high']
12
['Full_time_employed_high'] ['Full_time_employed_high']
inner loop 12
School_normal
School_normal
13
['Full_time_employed_very_high'] ['Full_time_employed_high']
14
['Full_time_employed_high'] ['Full_time_employed_high']
inner loop 14
School_close_Required_all
School_close_Required_partly
15
['Full_time_employed_low'] ['Full_time_employed_high']
16
['Full_time_employed_very_low'] ['Full_time_employed_high']
17
['Full_time_employed_extreme_low'] ['Full_time_employed_high']
18
['Full_time_employed_very_low'] ['Full_time_employed_high']
19
['Full_time_employed_very_low'] ['Full_time_employed_high']
20
['Full_time_employed_medium'] ['Full_time_employed_high']
['School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_normal', 'School_close_Required_all', 'School_close_Required_partly']
* -> input check
{'School_close_Required

GC_target:
['Full_time_employed_high'] [8]
local density
[[0.5], [0.0625], [0.1875]]
*
the ld dict: {'no_testing_policy': [0.5], 'open_public_testing': [0.0625], 'test_selected_ppl': [0.1875]}
density ratio:
[0.5]
[0.5714285714285714]
no_testing_policy 0.875
density ratio:
[0.0625]
[0.2857142857142857]
open_public_testing 0.21875
density ratio:
[0.1875]
[0.14285714285714285]
test_selected_ppl 1.3125
update the item candidate test_selected_ppl 0.1875
item local_density 0.1875 0.1875
test_selected_ppl has a relative importance: 1.0
the dr dict: {'no_testing_policy': 0.875, 'open_public_testing': 0.21875, 'test_selected_ppl': 1.3125}
the candidate :  {}
the confidence :  {}
* Exam
                       14
0      no_contact_tracing
1      no_contact_tracing
2      no_contact_tracing
3      no_contact_tracing
4      no_contact_tracing
5      no_contact_tracing
6      no_contact_tracing
7      no_contact_tracing
8      no_contact_tracing
9      no_contact_tracing
10     no_contact_tracing
1

3
['Full_time_employed_medium'] ['Full_time_employed_low']
4
['Full_time_employed_medium'] ['Full_time_employed_low']
5
['Full_time_employed_medium'] ['Full_time_employed_low']
6
['Full_time_employed_high'] ['Full_time_employed_low']
7
['Full_time_employed_medium'] ['Full_time_employed_low']
8
['Full_time_employed_high'] ['Full_time_employed_low']
9
['Full_time_employed_medium'] ['Full_time_employed_low']
10
['Full_time_employed_high'] ['Full_time_employed_low']
11
['Full_time_employed_very_high'] ['Full_time_employed_low']
12
['Full_time_employed_high'] ['Full_time_employed_low']
13
['Full_time_employed_very_high'] ['Full_time_employed_low']
14
['Full_time_employed_high'] ['Full_time_employed_low']
15
['Full_time_employed_low'] ['Full_time_employed_low']
inner loop 15
public_events_cancel_Required
public_events_cancel_Required
16
['Full_time_employed_very_low'] ['Full_time_employed_low']
17
['Full_time_employed_extreme_low'] ['Full_time_employed_low']
18
['Full_time_employed_very_low'

...

['Full_time_employed_low'] ['Full_time_employed_very_high']
16
['Full_time_employed_very_low'] ['Full_time_employed_very_high']
17
['Full_time_employed_extreme_low'] ['Full_time_employed_very_high']
18
['Full_time_employed_very_low'] ['Full_time_employed_very_high']
19
['Full_time_employed_very_low'] ['Full_time_employed_very_high']
20
['Full_time_employed_medium'] ['Full_time_employed_very_high']
['no_travel_control', 'no_travel_control', 'no_travel_control', 'highrisk_regions_ban']
* -> input check
{'border_closure': 7, 'highrisk_regions_ban': 1, 'no_travel_control': 13} ['no_travel_control', 'no_travel_control', 'no_travel_control', 'highrisk_regions_ban'] [2] ['Full_time_employed_very_high']
* ->C local count
['border_closure', 'highrisk_regions_ban', 'no_travel_control']
3
['border_closure', 'highrisk_regions_ban', 'no_travel_control']
[0, 1, 3]
GC_target:
['Full_time_employed_very_high'] [2]
local density
[[0.0], [0.25], [0.75]]
*
the ld dict: {'border_closure': [0.0], 'highrisk_

12
0
0
0
0
[[0.5714285714285714], [0.5714285714285714], [0.0], [0.0], [0.0], [0.0]]
target  ['Full_time_employed_very_high'] is list
2
0
2
0
0
0
0
0
0
creating the final candidate
the index []
[]
target: Full_time_employed_very_low
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
4
['Workplace_close_Recommended', 'Workplace_close_Required_partly', 'Workplace_close_all_essential_only', 'Workplace_normal']
3
['Public_events_normal', 'public_events_cancel_Recommended', 'public_events_cancel_Required']
3
['No_gatherings_restrictions', 'Restrictions_ten_ppl_less', 'Restrictions_tenppl_more']
2
['transport_closing_recommend', 'transport_normal']
3
['stay_home_not_require', 'stay_home_recommend', 'stay_home_required_exercise_only']
2
['no_movement_restrictions', 'restrict_movement']
3
['border_closur

[[1.0], [1.0]]
*
the ld dict: {'no_movement_restrictions': [1.0], 'restrict_movement': [1.0]}
density ratio:
[1.0]
[1.0]
no_movement_restrictions 1.0
density ratio:
[1.0]
[1.0]
restrict_movement 1.0
the dr dict: {'no_movement_restrictions': 1.0, 'restrict_movement': 1.0}
the candidate :  {}
the confidence :  {}
* Exam
                       9
0      no_travel_control
1      no_travel_control
2      no_travel_control
3      no_travel_control
4      no_travel_control
5      no_travel_control
6      no_travel_control
7      no_travel_control
8      no_travel_control
9      no_travel_control
10     no_travel_control
11     no_travel_control
12     no_travel_control
13  highrisk_regions_ban
14        border_closure
15        border_closure
16        border_closure
17        border_closure
18        border_closure
19        border_closure
20        border_closure
{'border_closure': 7, 'highrisk_regions_ban': 1, 'no_travel_control': 13}
['Full_time_employed_very_low'] as C
0
['Full_time_emplo

{15: 'gov_stringency_very_high', 17: 'ContainmentHealth_high'}
16 17
['Government_Response_high']
['ContainmentHealth_high']
your object is not a list, this is the initial L2
{16: 'Government_Response_high', 17: 'ContainmentHealth_high'}
1
1
1
1
2
2
0
0
3
3
2
2
3
3
2
2
0
0
0
0
2
2
1
1
1
1
2
2
2
2
0
0
1
1
0
0
1
1
0
0
0
0
2
2
2
2
0
0
1
1
2
2
0
0
0
0
0
0
2
2
2
2
0
0
1
1
0
0
2
2
1
1
1
1
6
6
1
1
3
3
4
4
2
2
2
2
2
2
1
1
3
3
4
4
2
2
2
2
2
2
3
3
1
1
0
0
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
[[0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.14285714285714285], [0.14285714285714285], [0.09523809523809523], [0.09523809523809523], [0.14285714285714285], [0.14285714285714285], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.0476190476190476

0
0
0
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
0
0
0
1
1
1
1
1
1
2
2
2
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
1
1
1
[[0.0], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619

In [23]:
fulltime.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)

,antecedent,consequent,density ratio,confidence
12,"[cases_huge_increase, ContainmentHealth_high]",Full_time_employed_very_low,17.500000,1.666667
2,"[cases_huge_increase, School_close_Required_all]",Full_time_employed_very_low,7.000000,1.333333
3,"[cases_increase, public_events_cancel_Required]",Full_time_employed_very_low,7.000000,1.333333
16,"[deaths_increase, Restrictions_ten_ppl_less]",Full_time_employed_very_low,7.000000,1.333333
0,cases_huge_increase,Full_time_employed_very_low,4.666667,1.333333
2,Government_Response_low,Full_time_employed_very_high,5.250000,1.000000
2,deaths_increase,Full_time_employed_very_low,3.000000,0.857143
3,School_close_Required_all,Full_time_employed_very_low,3.000000,0.857143
5,Restrictions_ten_ppl_less,Full_time_employed_very_low,3.000000,0.857143
10,Government_Response_high,Full_time_employed_very_low,3.000000,0.857143


In [24]:
DRBARMS_result_fulltime = fulltime.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)
DRBARMS_result_fulltime.to_csv(r"D:\Download\COVID19\DRBARMS_result_fulltime.csv", index = False, header=True)

In [25]:
unemployment = DRBARMS_algorithm(unemployment)

19
                             18
0      Unemployment_rate_medium
1         Unemployment_rate_low
2         Unemployment_rate_low
3         Unemployment_rate_low
4         Unemployment_rate_low
5         Unemployment_rate_low
6         Unemployment_rate_low
7         Unemployment_rate_low
8         Unemployment_rate_low
9         Unemployment_rate_low
10   Unemployment_rate_very_low
11   Unemployment_rate_very_low
12     Unemployment_rate_medium
13     Unemployment_rate_medium
14     Unemployment_rate_medium
15       Unemployment_rate_high
16       Unemployment_rate_high
17  Unemployment_rate_very_high
18  Unemployment_rate_very_high
19       Unemployment_rate_high
20     Unemployment_rate_medium
target: Unemployment_rate_high
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Required_all', 'School_close_Required_partly', 'School_normal']
4
['Workplace_clo

6
['Unemployment_rate_low'] ['Unemployment_rate_high']
7
['Unemployment_rate_low'] ['Unemployment_rate_high']
8
['Unemployment_rate_low'] ['Unemployment_rate_high']
9
['Unemployment_rate_low'] ['Unemployment_rate_high']
10
['Unemployment_rate_very_low'] ['Unemployment_rate_high']
11
['Unemployment_rate_very_low'] ['Unemployment_rate_high']
12
['Unemployment_rate_medium'] ['Unemployment_rate_high']
13
['Unemployment_rate_medium'] ['Unemployment_rate_high']
14
['Unemployment_rate_medium'] ['Unemployment_rate_high']
15
['Unemployment_rate_high'] ['Unemployment_rate_high']
inner loop 15
cover_less_than_half_income
cover_less_than_half_income
16
['Unemployment_rate_high'] ['Unemployment_rate_high']
inner loop 16
cover_less_than_half_income
cover_less_than_half_income
17
['Unemployment_rate_very_high'] ['Unemployment_rate_high']
18
['Unemployment_rate_very_high'] ['Unemployment_rate_high']
19
['Unemployment_rate_high'] ['Unemployment_rate_high']
inner loop 19
cover_less_than_half_income
cove

0
0
0
2
2
2
2
2
2
0
0
2
2
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
0
0
2
2
0
0
0
0
0
0
0
0
1
1
2
2
2
2
1
1
0
0
2
2
1
1
1
1
1
1
4
4
1
1
4
4
0
0
1
1
1
1
0
0
3
3
1
1
2
2
1
1
2
2
1
1
6
6
1
1
4
4
2
2
2
2
2
2
1
1
4
4
2
2
2
2
2
2
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
[[0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.04761904

0
0
2
2
2
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
1
1
1
2
2
2
1
1
1
2
2
2
1
1
1
[[0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.047619047619047616]]
target  ['Unemployment_rate_high

transport_normal
transport_normal
9
['Unemployment_rate_low'] ['Unemployment_rate_low']
inner loop 9
transport_normal
transport_normal
10
['Unemployment_rate_very_low'] ['Unemployment_rate_low']
11
['Unemployment_rate_very_low'] ['Unemployment_rate_low']
12
['Unemployment_rate_medium'] ['Unemployment_rate_low']
13
['Unemployment_rate_medium'] ['Unemployment_rate_low']
14
['Unemployment_rate_medium'] ['Unemployment_rate_low']
15
['Unemployment_rate_high'] ['Unemployment_rate_low']
16
['Unemployment_rate_high'] ['Unemployment_rate_low']
17
['Unemployment_rate_very_high'] ['Unemployment_rate_low']
18
['Unemployment_rate_very_high'] ['Unemployment_rate_low']
19
['Unemployment_rate_high'] ['Unemployment_rate_low']
20
['Unemployment_rate_medium'] ['Unemployment_rate_low']
['transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 'transport_normal', 't

...

0
0
0
2
2
1
1
2
2
1
1
2
2
1
1
[[0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.047619047619047616], [0.047619047619047616]]
target  ['Unemployment_rate_medium'] is list
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
4
0
1
1
0
0
2
0
2
2
0
0
4
0
1
1
0
0
2
0
2
2
0
0
4
0
1
1
0
0
2
creating the final candidate
the index []
[]
target: Unemployment_rate_very_high
4
['cases_huge_increase', 'cases_increase', 'cases_small_increase', 'no_cases']
4
['deaths_huge_increase', 'deaths_increase', 'deaths_small_increase', 'no_deaths']
3
['School_close_Required_all', 'School_close_Required_pa

9
['Unemployment_rate_low'] ['Unemployment_rate_very_high']
10
['Unemployment_rate_very_low'] ['Unemployment_rate_very_high']
11
['Unemployment_rate_very_low'] ['Unemployment_rate_very_high']
12
['Unemployment_rate_medium'] ['Unemployment_rate_very_high']
13
['Unemployment_rate_medium'] ['Unemployment_rate_very_high']
14
['Unemployment_rate_medium'] ['Unemployment_rate_very_high']
15
['Unemployment_rate_high'] ['Unemployment_rate_very_high']
16
['Unemployment_rate_high'] ['Unemployment_rate_very_high']
17
['Unemployment_rate_very_high'] ['Unemployment_rate_very_high']
inner loop 17
cover_less_than_half_income
cover_less_than_half_income
18
['Unemployment_rate_very_high'] ['Unemployment_rate_very_high']
inner loop 18
cover_less_than_half_income
cover_less_than_half_income
19
['Unemployment_rate_high'] ['Unemployment_rate_very_high']
20
['Unemployment_rate_medium'] ['Unemployment_rate_very_high']
['cover_less_than_half_income', 'cover_less_than_half_income', 'cover_less_than_half_income'

0
0
0
0
2
2
2
2
0
0
1
1
2
2
0
0
0
0
0
0
0
0
0
0
2
2
2
2
0
0
1
1
2
2
0
0
0
0
0
0
0
0
2
2
2
2
0
0
1
1
0
0
2
2
1
1
1
1
1
1
1
1
2
2
1
1
1
1
0
0
2
2
2
2
6
6
1
1
3
3
4
4
2
2
2
2
2
2
1
1
3
3
4
4
2
2
2
2
2
2
3
3
1
1
0
0
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
[[0.047619047619047616], [0.047619047619047616], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.047619047619047616], [0.047619047619047616], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.09523809523809523], [0.0], [0.0], [0.047619047619047616], [0.04761904

3
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
4
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
5
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
6
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
7
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
8
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
9
['Unemployment_rate_low'] ['Unemployment_rate_very_low']
10
['Unemployment_rate_very_low'] ['Unemployment_rate_very_low']
inner loop 10
No_gatherings_restrictions
No_gatherings_restrictions
11
['Unemployment_rate_very_low'] ['Unemployment_rate_very_low']
inner loop 11
No_gatherings_restrictions
No_gatherings_restrictions
12
['Unemployment_rate_medium'] ['Unemployment_rate_very_low']
13
['Unemployment_rate_medium'] ['Unemployment_rate_very_low']
14
['Unemployment_rate_medium'] ['Unemployment_rate_very_low']
15
['Unemployment_rate_high'] ['Unemployment_rate_very_low']
16
['Unemployment_rate_high'] ['Unemployment_rate_very_low']
17
['Unemp

{0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}}
0 1
[]
[]
0 2
[]
[]
0 3
[]
[]
0 4
[]
[]
0 5
[]
[]
0 6
[]
[]
0 7
[]
[]
0 8
[]
[]
0 9
[]
[]
0 10
[]
[]
0 11
[]
[]
0 12
[]
[]
0 13
[]
[]
0 14
[]
[]
0 15
[]
[]
0 16
[]
[]
0 17
[]
[]
1 2
[]
[]
1 3
[]
[]
1 4
[]
[]
1 5
[]
[]
1 6
[]
[]
1 7
[]
[]
1 8
[]
[]
1 9
[]
[]
1 10
[]
[]
1 11
[]
[]
1 12
[]
[]
1 13
[]
[]
1 14
[]
[]
1 15
[]
[]
1 16
[]
[]
1 17
[]
[]
2 3
[]
[]
2 4
[]
[]
2 5
[]
[]
2 6
[]
[]
2 7
[]
[]
2 8
[]
[]
2 9
[]
[]
2 10
[]
[]
2 11
[]
[]
2 12
[]
[]
2 13
[]
[]
2 14
[]
[]
2 15
[]
[]
2 16
[]
[]
2 17
[]
[]
3 4
[]
[]
3 5
[]
[]
3 6
[]
[]
3 7
[]
[]
3 8
[]
[]
3 9
[]
[]
3 10
[]
[]
3 11
[]
[]
3 12
[]
[]
3 13
[]
[]
3 14
[]
[]
3 15
[]
[]
3 16
[]
[]
3 17
[]
[]
4 5
[]
[]
4 6
[]
[]
4 7
[]
[]
4 8
[]
[]
4 9
[]
[]
4 10
[]
[]
4 11
[]
[]
4 12
[]
[]
4 13
[]
[]
4 14
[]
[]
4 15
[]
[]
4 16
[]
[]
4 17
[]
[]
5 6
[]
[]
5 7
[]
[]
5 8
[]
[]
5 9
[]
[]
5 10
[]
[]
5 11
[]
[]
5 12
[]
[]
5 

In [28]:
#result2 = 
unemployment.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)

,antecedent,consequent,density ratio,confidence
10,"[School_close_Required_all, Workplace_close_Re...",Unemployment_rate_high,17.50,1.666667
17,"[School_close_Required_all, stay_home_recommend]",Unemployment_rate_high,10.50,1.000000
3,"[deaths_increase, comprehensive_tracing]",Unemployment_rate_very_high,10.50,1.000000
8,"[deaths_increase, Restrictions_ten_ppl_less]",Unemployment_rate_high,5.25,1.000000
12,"[School_close_Required_all, public_events_canc...",Unemployment_rate_high,5.25,1.000000
...,...,...,...,...
7,"[deaths_increase, Restrictions_ten_ppl_less]",Unemployment_rate_high,1.75,0.333333
9,"[deaths_increase, gov_stringency_high]",Unemployment_rate_high,1.75,0.333333
2,"[deaths_increase, Workplace_close_Required_par...",Unemployment_rate_high,1.75,0.333333
3,"[deaths_increase, Workplace_close_Required_par...",Unemployment_rate_high,1.75,0.333333


In [29]:
DRBARMS_result_unemployment = unemployment.sort_values(by=[ 'confidence', 'density ratio'], ascending=False)
DRBARMS_result_unemployment.to_csv(r"D:\Download\COVID19\DRBARMS_result_unemployment.csv", index = False, header=True)